In [139]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from typing import Tuple
import boto3 as boto
import random
import _pickle as cPickle
from torch.utils.data.distributed import DistributedSampler
import os
import torchvision.transforms as T 
from torchvision.transforms import v2
from torchvision.io import read_image

In [140]:
read_image("patient1.png").float()

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [234., 234., 234.,  ..., 234., 234., 234.],
         [234., 234., 234.,  ..., 234., 234., 234.],
         ...,
         [234., 234., 234.,  ..., 234., 234., 234.],
         [234., 234., 234.,  ..., 234., 234., 234.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [247., 247., 247.,  ..., 247., 247., 247.],
         [247., 247., 247.,  ..., 247., 247., 247.],
         ...,
         [247., 247., 247.,  ..., 247., 247., 247.],
         [247., 247., 247.,  ..., 247., 247., 247.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [255., 255., 255.,  ..., 255., 255., 255.],
         [255., 255., 255.,  ..., 255., 255., 255.],
         ...,
         [255., 255., 255.,  ..., 255., 255., 255.],
         [255., 255., 255.,  ..., 255., 255., 255.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

In [283]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import os
import pydicom as dicom
import math
import time

class PositionalEncoding(nn.Module):
        
    def __init__(self, data_shape, dropout = .1):
        super(PositionalEncoding,self).__init__()
        
        #Get data shape
        self.in_channels, self.row_len, self.col_len = data_shape
        
        self.learned_embedding = nn.Parameter(torch.zeros(data_shape))
        
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self,data):
        
        data = data + self.learned_embedding
        data = self.dropout(data)
        
        return data

class Convlayer(nn.Module):
    
    def __init__(self,data_shape:tuple,num_patches:int,output_dim:int = None):
        super(Convlayer,self).__init__()
        self.num_patches = num_patches
        
        self.batch,self.in_channels, self.row_len, self.col_len = data_shape
        
        assert self.row_len % num_patches == 0 
        assert self.col_len % num_patches == 0
        
        
        self.patch_row = self.row_len // num_patches
        self.patch_col = self.col_len // num_patches
        
        self.embed_dim = int(self.in_channels * self.patch_row * self.patch_col)
        
        self.kernel_len = (int(self.patch_row), int(self.patch_col))
        
        patch_area = int(self.row_len * self.col_len)
        
        self.conv2d_1 = nn.Conv2d(in_channels = self.in_channels, 
                                  out_channels = self.embed_dim, 
                                  kernel_size =self. kernel_len, 
                                  stride = self.kernel_len)
        
        
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten(start_dim=2) 
        if output_dim == None:
            self.dnn = nn.Linear(self.embed_dim,self.embed_dim)
        else:
            self.output_dim = output_dim
            self.dnn = nn.Linear(self.embed_dim,output_dim)
    def forward(self,data):
        
        
        #x = self.conv2d_1(data)
        #print(x.shape)
        #x = self.relu(x)
        #x = self.flatten(x)
        #print(x.shape)
        #x = torch.transpose(x,1,2)
        print(f'DATA SHAPE: {data.shape}')
        patches = data.unfold(2,self.row_len,self.row_len).unfold(3,self.col_len,self.col_len)
        patches = torch.reshape(patches,(self.batch,self.num_patches**2,self.embed_dim))
        
        #print(x.shape)
        x = self.dnn(patches)
        
        return x
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self,data_shape,num_heads):
        
        super(MultiHeadAttention,self).__init__()
        self.batch, self.patch, self.embed = data_shape
        self.attn = nn.MultiheadAttention(self.embed,num_heads,batch_first = True)
    def forward(self,data):
        outputs , _ = self.attn(query=data, key=data, value=data, need_weights = False)
        return outputs

class MLP(nn.Module):
    def __init__(self,data_shape,output_size,dropout = .1):
        super(MLP,self).__init__()
        self.batch, self.patch, self.embed = data_shape
        hidden_output = self.embed * 2
        self.lnn1 = nn.Linear(self.embed, hidden_output)
        self.dropout1 = nn.Dropout(dropout)
        self.fnn2 = nn.Linear(hidden_output, output_size)
        self.dropout2 = nn.Dropout(dropout)
        self.gelu = nn.GELU()
    def forward(self,data):
        
        x = self.lnn1(data)
        x = self.gelu(x)
        x = self.dropout1(x)
        x = self.fnn2(x)
        x = self.gelu(x)
        x = self.dropout2(x)
        
        return x
        
class TransformerEncoder(nn.Module):
    
    def __init__(self,data_shape,num_heads,dropout=.1):
        super(TransformerEncoder,self).__init__()
        self.data_shape = data_shape
        self.batch, self.patch, self.embed = data_shape
        self.ln1 = nn.LayerNorm([self.patch,self.embed])
        self.ln2 = nn.LayerNorm([self.patch,self.embed])
        self.MHA = MultiHeadAttention(data_shape,num_heads)
        self.mlp = MLP(data_shape, output_size = self.embed, dropout=dropout)
        
    def forward(self,data):
        
        x = self.ln1(data)
        att_out = self.MHA(x)
        att_out = att_out + data
        after_ln2 = self.ln2(att_out)
        after_ln2 = self.mlp(after_ln2)
        after_ln2 = after_ln2 + att_out
        
        return after_ln2
        
class VisionTransformer(nn.Module):
    def __init__(self,data_shape,num_heads,num_layers = 6,dropout = .1):
        super(VisionTransformer,self).__init__()
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(
                f'{i}', TransformerEncoder(data_shape=data_shape,num_heads = num_heads,dropout = dropout))
    
    def forward(self,data):
        x = data
        for blk in self.blks:
            x = blk(x)
        return x
    
class ClassificationHead(nn.Module):
    def __init__(self,input_layer,hidden_layer_1,hidden_layer_2,hidden_layer_3,num_output,dropout=.1):
        super(ClassificationHead,self).__init__()
        self.ln1 = nn.LayerNorm(input_layer)
        self.fnn1 = nn.Linear(input_layer,hidden_layer_1)
        self.dropout_1 = nn.Dropout(dropout)
        self.ln2 = nn.LayerNorm(hidden_layer_1)
        self.fnn2 = nn.Linear(hidden_layer_1,hidden_layer_2)
        self.dropout_2 = nn.Dropout(dropout)
        self.ln3 = nn.LayerNorm(hidden_layer_2)
        self.fnn3 = nn.Linear(hidden_layer_2,hidden_layer_3)
        self.dropout_3 = nn.Dropout(dropout)
        self.fnn4 = nn.Linear(hidden_layer_3,num_output)
        
    def forward(self,data):
        x = self.ln1(data)
        x = self.fnn1(x)
        x = self.dropout_1(x)
        x = self.ln2(x)
        x = self.fnn2(x)
        x = self.dropout_2(x)
        x = self.ln3(x)
        x = self.fnn3(x)
        x = self.dropout_3(x)
        x = self.fnn4(x)
        
        return x
    
class PRPModel(nn.Module):
    def __init__(self,
                 data_shape,
                 num_patch:int,
                 num_heads,
                 num_output,
                 num_layers,
                 conv_output_dim,
                 hidden_layer_1,
                 hidden_layer_2,
                 hidden_layer_3,
                dropout = .1):
        super(PRPModel,self).__init__()
        self.batch_size = data_shape[0]
        self.data_shape = data_shape[1:]
        
        self.conv_layer = Convlayer(data_shape,num_patch,conv_output_dim)
        self.in_channels, self.row_len, self.col_len = self.data_shape
        assert self.row_len % num_patch == 0 
        assert self.col_len % num_patch == 0
        
        patch_row = self.row_len // num_patch
        patch_col = self.col_len // num_patch
        
        embed_dim = patch_row * patch_col * self.in_channels
        assert embed_dim % num_heads == 0, f"embed_dimension is not divisible by num_heads \nembed_dim: {embed_dim},heads:{num_heads}"
        
        self.data_shape = (self.batch_size,num_patch**2,patch_row * patch_col * self.in_channels)
        
        self.pos_encode = PositionalEncoding(self.data_shape,dropout)
        
        
        self.visual_transformer = VisionTransformer(self.data_shape,num_heads,num_layers,dropout)
        
        self.input_layer = self.data_shape[1] * self.data_shape[2]
        
        self.ClassificationHead = ClassificationHead(self.input_layer,
                                                     hidden_layer_1,
                                                     hidden_layer_2,
                                                     hidden_layer_3,
                                                     num_output,
                                                     dropout=.1)
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self,data):
        x = self.conv_layer(data)
        x = self.pos_encode(x)
        x = self.visual_transformer(x)
        x = torch.reshape(x,(self.batch_size,self.input_layer))
        x = self.ClassificationHead(x)
        print(f"BEFORE SOFTMAX: {x}")
        x = self.softmax(x)
        #x = torch.argmax(x,axis = 1)
        return x

In [325]:
#Not set up for multi-gpu running, need to add things to get that ready 
#Also would like to add confusion matrix output
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import pandas as pd
import _pickle as cPickle
import random
import itertools
import time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import boto3 as boto
import torch.multiprocessing as mp


#class Trainer:
    
    #def __init___(self, 
                  #model : torch.nn.Module,
                  #optimizer: torch.optim.Optimizer,
                  #loss_fn: torch.nn,
                  #save_interval: int, 
                  #metric_interval: int,
                  #train_data: DataLoader,
                  #validation_data: DataLoader = None, 
                  #test_data: DataLoader = None,
                  #save_path: str = None): 
        
        #Setting all variables equal to a local counterpart 
        #self.model = model
        #self.optimizer = optimizer
        #self.loss_fn = loss_fn
        #self.save_interval = save_interval
        #self.metric_interval = metric_interval
        #self.train_data = train_data
        #self.validation_data = validation_data
        #self.test_data = test_data
        #self.save_path = save_path
        
        #going to be used in evaluating function to decrease latency of model 
        #self.curr_predictions = []
        #self.curr_labels = []
        
class Trainer:
    def __init__(self,
                 model: torch.nn.Module,
                 optimizer: torch.optim.Optimizer,
                 loss_fn: torch.nn.Module,
                 save_interval: int,
                 metric_interval: int,
                 train_data: DataLoader,
                 validation_data: DataLoader = None,
                 test_data: DataLoader = None,
                 save_path: str = None):
        
        # Setting all variables equal to a local counterpart
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.save_interval = save_interval
        self.metric_interval = metric_interval
        self.train_data = train_data
        self.validation_data = validation_data
        self.test_data = test_data
        self.save_path = save_path

        # Going to be used in evaluating function to decrease latency of model
        self.curr_predictions = []
        self.curr_labels = []
        
    def _run_batch(self,batch: torch.Tensor, batch_labels: torch.Tensor):
        #Setting current gradient to zero for new batch
        self.optimizer.zero_grad()
        #Running model on current batch
        print("running _run_batch")
        pred_output = self.model(batch)
        print("Done running batch")
        #Appending predicted values to list for evaluation 
        self.curr_predictions.append(pred_output)
        #Appending label values to list for evaluation
        self.curr_labels.append(batch_labels)
        
        #Computing loss 
        print(f"PRED OUTPUT: {pred_output}")
        print(f"BATCH LABEL: {batch_labels}")
        loss = self.loss_fn(pred_output.float(),batch_labels.float())
        #Computing gradient for each parameter in model
        loss.backward()
        #Gradient descent 
        self.optimizer.step()
        
    def _run_epoch(self, epoch:int):
        #Including epoch num in init bc likely will want to print out at somepoint to see how quickly model runs
        #Setting model to train
        
        self.model.train()
        #Re-initiating prediction/label accumulator for evaluation of specific epoch
        
        self.curr_predictions = []
        self.curr_labels = []
        
        #Looping over each training batch
        print("training model")
        for batch_tensor, batch_labels in self.train_data:
            
            print(f"batch_tensor: {batch_tensor.shape}")
            print(f"batch_labels: {batch_labels.shape}")
            
            #Running gradient descent on each batch
            self._run_batch(batch_tensor,batch_labels)
        print("done training model")
        print("exiting _run_epoch")
    
    #TODO: FINISH how to save to server
    def _save_checkpoint(self, epoch: int):
        #Getting the model weights at particular checkpoint
        print("in save_checkpoint method")
        checkpoint_model = self.model.state_dict()
        print("after getting the state-dict")
        #Pickling model into checkpoint_{epoch} file
        #Note that pickle.dump saves model in local directory
        #Need to delete after dump and upload
        torch.save(checkpoint_model,f'checkpoint_{epoch}.pt')
        #cPickle.dump(checkpoint_model, open(f'checkpoint_{epoch}.pt', 'wb'))
        print("after pickle dump")
        #NEED TO FINISH SAVING TO FOLDER OF DICTIONARIES
        
    
    def train(self, num_epochs:int):
        
        #Looping over number of epochs
        for epoch in range(1,num_epochs+1):
            
            #Running an epoch
            print(f"running {epoch} epoch")
            self._run_epoch(epoch)
            
            print("outside self.save_interval")
            #Code to save the model every save_interval   
            if self.save_interval > 0 and epoch % self.save_interval == 0:
                print("In save_interval")
                self._save_checkpoint(epoch)
            #Saving the last model
            elif epoch == num_epochs:
                self._save_checkpoint(epoch)

            #Evaluating model every metric_interval
            print("outside self.metric_interval")
            if self.metric_interval > 0 and epoch % self.metric_interval == 0:
                #Decreases time bc saved inferences for training data in list
                #Evaluating Training set
                self._evaluate(None)
                #Will have to do inference for test set
                if self.test_data != None:
                    #Evaluating test set
                    self._evaluate(self.test_data)
                    #Resetting the model to train 
                    self.model.train()
                
    def _evaluate(self, dataloader: DataLoader = None):
        #Converting to torch.no_grad to prevent gradient calculation
        with torch.no_grad():
            #Set model to evaluation
            self.model.eval()
            #If dataloader none, it means we are looking at the current training set accuracy 
            if dataloader == None:
                #Using already predicted values that we accumulated during the training 
                #This obviously is a lower bound on the accuracy of our model on the training set
                #However, transformer latency is extremely large so this will decrease training time overall
                #Also we don't care about the actual training accuracy, we only care about the overall trends of 
                #training accuracy
                predict_output = torch.vstack(self.curr_predictions)
                labels = torch.vstack(self.curr_labels)
                print("/tTRAINING SET VALUES")
            else:
                #Creating accumulators for test set
                test_predict = []
                test_labels = []
                #Looping over each tensor and label in the dataloader
                for batch_tensor, batch_label in dataloader:
                    #Predicting using model on test set
                    prediction = self.model(batch_tensor)
                    #accumulating model predictions and labels of test set
                    test_predict.append(prediction)
                    test_labels.append(batch_label)
                #Vstacking outputs and labels so that tensors read (patient x 1)
                #Note loss function is MSE, so output from model will be a singular value that relates to our 
                #actual scale
                #This differs from CrossEntropyLoss, where model output would be vector of length (num_classes)
                #and each entry would be a probability of particular class
                predict_output = torch.vstack(test_predict)
                labels = torch.vstack(test_labels)
                print("/tTEST SET VALUES")
            
            #Squeezing output to get rid of nested tensors
            predict_output = torch.squeeze(predict_output)
            labels = torch.squeeze(labels)
            
            predict_output = torch.argmax(predict_output,axis = 1)
            labels = torch.argmax(labels,axis = 1)
            #Calculating loss of the model for train/test set
            loss = self.loss_fn(predict_output.float(), labels.float())
            #Calculating Mean Absolute Error based on train/test set
            print(f"PREDICT_OUTPUT: {predict_output}")
            print(f"LABELS: {labels}")
            MAE = (predict_output.float() - labels.float()).abs().mean().item()
            
            #Rounding predicted output so that it matches the exact categories given by Norwood scale
            #predict_output = torch.round(predict_output)
            
            #Calculating how many predictions were correct
            num_correct = (predict_output == labels).sum().item()
            
            #Calculating accuracy of model
            acc = num_correct / len(labels)
            
            print(f"\t\t NUMBER CORRECT: {num_correct}")
            print(f"\t\t ACCURACY: {acc}")
            print(f"\t\t MEAN ABSOLUTE ERROR: {MAE}")
            print(f"\t\t LOSS: {loss}")
            print(f"\t\t PREDICTED: {predict_output}")
            print(f"\t\t LABELS: {labels}")
            print(f"++++++++++++++++++++++++++++++++++++++++++++++++++++")                     

In [326]:
#SINGLE GPU INSTANCE CAPABILITIES ONLY. WILL HAVE TO UPDATE FOR MULTIGPU
class PRPDataSet(Dataset):
    
    def __init__(self, patient_ids: list,patient_labels: dict, num_classification_output: int,path: str, size: tuple):
        
        #dictionary of where patient ids are keys and rating is value
        self.patient_labels = patient_labels
        
        #list of patient ids
        self.patient_ids = patient_ids
        
        #path
        self.path = path
        
        #calculating mean and standard deviation along channels for entire dataset
        mean,std = self.mean_std()
        
        self.num_classification_output = num_classification_output
        
        #transformation used on each image 
        #Note that mean/std are vectors of len(4) for the alpha,RBG channels
        self.transforms = v2.Compose([
            v2.Resize(size),
            v2.ToImage(), 
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean = mean,std = std)
        ])
        
    def __len__(self):
        return len(self.patient_ids)
    
    def __getitem__(self,idx:int):
        
        #indexing via list
        patient = self.patient_ids[idx]
        
        #pulling image from storage bucket and un-pickling it
        #THIS MAY HAVE TO BE CHANGED DEPENDING ON HOW EXACTLY THE IMAGES ARE STORED
        #image = cPickle.load(open(f'{self.path}/{patient}/image'),"rb")
        image = read_image(f'{patient}.png')
        
        #Resize image
        resized_image = self.transforms(image)
        
        #Get label
        label_tensor = [0 for _ in range(self.num_classification_output)]
        label = torch.tensor(self.patient_labels[patient])
        label_tensor[label] = 1
        label_tensor = torch.tensor(label_tensor)
        
        return resized_image, label_tensor
    
    #Should maybe try to calculate mean_std deviation once prior to running script and store data
    #So that we do not have to rerun this step everytime the script is rerun 
    #
    #Note we are calculating the mean of pixel average for each the image
    #And the mean of the pixel std. dev for each the images
    def mean_std(self):
        #sum accumulator tensor for 4 channels x number of patients
        sum_values = torch.zeros(4,len(self.patient_ids))
        #std. dev accumulator tensor for 4 channels x number of patients
        std_values = torch.zeros(4,len(self.patient_ids))
        #Looping over photos
        #Have to do this, cannot vectorize since the images are all of different shapes
        for i in range(len(self.patient_ids)):
            #Getting patient
            patient = self.patient_ids[i]
            #opening photo from path
            #Make sure to pickle these images as Tensor objects
            #image = cPickle.load(open(f'{patient}.png',"rb"))
            image = read_image(f'{patient}.png').float()
            #Getting pixel average of the photo
            pixel_avg = torch.mean(image, dim = (1,2))
            #Storing patient i in ith row of sum value
            sum_values[:,i] = pixel_avg
            #Getting pixel std. dev of the photo
            pixel_std = torch.std(image, dim = (1,2))
            #Storing patient i std. dev ith row of std. dev matrix
            std_values[:,i] = pixel_std
        
        #Calculating average mean and std. dev. values
        a = torch.mean(sum_values, axis = 1)
        b = torch.mean(std_values, axis = 1)
        
        return a,b
    
def sequential_train_test_split(split: tuple, labels:dict):
    
    #Getting number of patients 
    num_patients = len(labels.keys())
    #Getting how patients to include in training
    training_num = int(split[0] * num_patients)
    #creating training set
    training_patients = labels.keys()[:training_num]
    #Creating Test set 
    testing_patients = labels.keys()[training_num:]

    return training_patients, testing_patients

def random_train_test_split(split: tuple, labels:dict):
    
    #Converting label list to label set to use subtraction
    patient_set = set(labels.keys())
    
    #Traning set
    training_patients = set()

    #calculating number of patients
    num_patients = len(labels.keys())
    
    #calculating number of patients in training set
    training_num = int(split[0] * num_patients)
    
    
    i = 0
    #looping number of patients 
    while i != training_num:
        #Choosing a patient from the list of left over patients not yet chosen
        curr_patient = random.choice(list(patient_set-training_patients))
        #adding patient to list of training patient
        training_patients.add(curr_patient)
        #Iterating
        i += 1

    #Getting test set
    test_patients = patient_set - training_patients

    #Reconverting back to list 
    return list(training_patients), list(test_patients)

def get_train_test_dataset(dict_path: str, data_path: str, num_class_output: int,size: tuple ,sequential: bool, split:tuple):
    
    #Getting dictionary that has patient keys and scores as values
    #Path may need to be changed
    patient_labels = cPickle.load(open(dict_path, 'rb'))
    
    #If want the data to be split sequentially (i.e in order)
    if sequential:
        train, test = sequential_train_test_split(split,patient_labels)
    else:
    #If want data split randomly (More Likely used)
        train, test = random_train_test_split(split,patient_labels)
    
    #Creating PRPDataSet objects for both train and test sets
    train_set = PRPDataSet(train,patient_labels,num_class_output,data_path,size)
    test_set = PRPDataSet(train,patient_labels,num_class_output,data_path,size)
    
    return train_set,test_set

def PRPDataLoader(dict_path: str, data_path: str, num_class_output: int,size: tuple,sequential: bool, split: tuple, batch: int):
    
    #Getting PRPDataSet objects for both rain and test sets 
    train_set, test_set = get_train_test_dataset(dict_path, data_path, num_class_output, size ,sequential, split)
    
    #Creating DataLoader with Train and Set Data sets
    train_generator = DataLoader(train_set, batch_size = batch, shuffle = True)
    test_generator = DataLoader(test_set, batch_size = batch, shuffle = True)
    
    return train_generator,test_generator

In [347]:
patient_list = ["patient1","patient2","patient3"]
test_dict = cPickle.load(open("test_dict","rb"))
data = PRPDataSet(patient_list,test_dict,5,"",(200,200))

In [348]:
train,test = PRPDataLoader("test_dict","",5,(200,200),False,(1,0),3)

In [349]:
for i,j in train:
    print(i.shape)
    print(j.shape)

torch.Size([3, 4, 200, 200])
torch.Size([3, 5])


In [350]:
next(iter(train))[1].shape

torch.Size([3, 5])

In [357]:
model = PRPModel((3,4,200,200),10,10,5,2,None,100,100,100)

In [358]:
adam_optimizer = torch.optim.Adam(
        model.parameters(), lr=0.0001, weight_decay=0.0001)
    
mse_loss = torch.nn.MSELoss()

In [359]:
trainer = Trainer(model,
                  adam_optimizer,
                  mse_loss,
                  100,
                  1,
                  train_data = train,
                  test_data= test,
                  save_path = "")

In [360]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

64206805

In [361]:
trainer.train(2)

running 1 epoch
training model
batch_tensor: torch.Size([3, 4, 200, 200])
batch_labels: torch.Size([3, 5])
running _run_batch
DATA SHAPE: torch.Size([3, 4, 200, 200])
BEFORE SOFTMAX: tensor([[ 0.1041, -0.6889, -0.4344, -0.7440,  0.0172],
        [ 0.2236, -0.4966, -0.2651, -0.2668, -0.2064],
        [ 0.2534, -0.6150, -0.2541, -1.0031, -0.0309]],
       grad_fn=<AddmmBackward0>)
Done running batch
PRED OUTPUT: tensor([[0.2958, 0.1338, 0.1726, 0.1267, 0.2711],
        [0.2974, 0.1447, 0.1824, 0.1821, 0.1934],
        [0.3269, 0.1372, 0.1968, 0.0931, 0.2460]], grad_fn=<SoftmaxBackward0>)
BATCH LABEL: tensor([[0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0]])
done training model
exiting _run_epoch
outside self.save_interval
outside self.metric_interval
/tTRAINING SET VALUES
PREDICT_OUTPUT: tensor([0, 0, 0])
LABELS: tensor([4, 3, 2])
		 NUMBER CORRECT: 0
		 ACCURACY: 0.0
		 MEAN ABSOLUTE ERROR: 3.0
		 LOSS: 9.666666984558105
		 PREDICTED: tensor([0, 0, 0])
		 LABELS: tenso

In [337]:
test_dict = {"patient1" : 2, "patient2" : 3,"patient3":4}
cPickle.dump(test_dict, open(f'test_dict', 'wb'))

In [282]:
x = torch.zeros(2,4,200,200)
patches = x.unfold(2, 20, 20)
print(patches.shape)
patches = patches.unfold(3, 20, 20)

patches = torch.reshape(patches,(2,100,4 * 20 * 20))

torch.Size([2, 4, 10, 200, 20])


In [266]:
patches.shape

torch.Size([100, 1600])

In [101]:
practice.unfold(1,20,20).shape

torch.Size([4, 10, 200, 20])